In [ ]:
"2023-12-20-Ratemap Center of Mass Returning Coordinates Not Positions.ipynb"


In [ ]:
# Problem: I discovered that `Ratemap`'s `ratemap.peak_tuning_curve_center_of_masses` property was specified in bin-indicies instead of actual x-positions

ratemap

# Solution: I redefined `tuning_curve_CoM_coordinates = deepcopy(ratemap.peak_tuning_curve_center_of_masses)` to use the `self.xbin` to determine the actual positions via interpolation.


In [ ]:
    @property
    def peak_tuning_curve_center_of_mass_bin_coordinates(self) -> NDArray:
        """ returns the coordinates (in bin-index space) of the center of mass of each of the tuning curves."""
        return compute_placefield_center_of_mass_coord_indicies(self.pdf_normalized_tuning_curves) # in coordinate (index) space
    
    
    @property
    def peak_tuning_curve_center_of_masses(self) -> NDArray:
        """ returns the locations of the center of mass of each of the tuning curves."""
        tuning_curve_CoM_coordinates = self.peak_tuning_curve_center_of_mass_bin_coordinates # in coordinate (index) space
        assert self.ndim == 1, f"tuning-curve CoM currently only implemented for 1D ratemap."
        xbin = self.xbin.copy()
        xbin_edge_labels = np.arange(len(xbin)) # the index range spanning all x-bins
        assert np.all(np.diff(xbin) > 0), f"requires monotonically increasing bins"
        assert np.allclose(np.diff(xbin), np.full_like(np.diff(xbin), np.diff(xbin)[0])), f'Requres equally spaced bins'
        tuning_curve_CoM_positions = np.interp(tuning_curve_CoM_coordinates, xp=xbin_edge_labels, fp=xbin) # in position space
        return tuning_curve_CoM_positions

In [ ]:
# Fixed issue with Ratemap's `.peak_tuning_curve_center_of_masses` returning index-space coordinates not real positions.
# Renamed `compute_placefield_center_of_masses` to `compute_placefield_center_of_mass_coord_indicies`
def compute_placefield_center_of_mass_coord_indicies(tuning_curves: NDArray) -> NDArray:
    """ returns the coordinates (index-space, not track-position space) of the center of mass for each of the tuning_curves. """
    return np.squeeze(np.array([ndimage.center_of_mass(x) for x in tuning_curves]))

In [ ]:
from scipy.interpolate import interp1d
from scipy.ndimage import gaussian_filter1d

def map_CoM_to_position_interp_array(xbin, tuning_curve_CoM_coordinates, xbin_edge_labels) -> NDArray:
  """
  Maps center-of-mass coordinates back to actual positions as a flat numpy array.

  Args:
      xbin: np.array, the bin centers (not edges).
      tuning_curve_CoM_coordinates: np.array, the center-of-mass coordinates for each tuning curve.
      xbin_edge_labels: np.array, the bin edge labels (not center).

  Returns:
      positions: np.array, the estimated positions corresponding to the input center-of-mass values.
  """
  positions = np.full_like(tuning_curve_CoM_coordinates, np.nan)
  for i, CoM in enumerate(tuning_curve_CoM_coordinates):
    # Find the bin containing the center-of-mass
    bin_idx = (xbin_edge_labels <= CoM).sum() - 1
    
    # Check if CoM falls within the valid range
    if bin_idx == 0 or bin_idx == len(xbin) - 1:
      continue  # Skip invalid values and leave them as NaN

    # Interpolate between bin edges
    interpolator = interp1d(xbin_edge_labels[bin_idx:bin_idx+2], xbin[bin_idx:bin_idx+2], kind="linear")
    positions[i] = interpolator(CoM)
  return positions

def map_CoM_to_position_weighted_average(xbin, tuning_curve_CoM_coordinates, xbin_edge_labels):
  """
  Maps center-of-mass coordinates back to actual positions using inverse weighted average.

  Args:
      xbin: np.array, the bin centers (not edges).
      tuning_curve_CoM_coordinates: np.array, the center-of-mass coordinates for each tuning curve.
      xbin_edge_labels: np.array, the bin edge labels (not center).

  Returns:
      positions: np.array, the estimated positions corresponding to the input center-of-mass values.
  """
  positions = np.full_like(tuning_curve_CoM_coordinates, np.nan)
  for i, CoM in enumerate(tuning_curve_CoM_coordinates):
    # Find contributing bins
    bin_idx1 = (xbin_edge_labels < CoM).sum() - 1
    bin_idx2 = (xbin_edge_labels >= CoM).sum() - 1
    
    # Calculate weighted average if valid range
    if bin_idx1 != -1 and bin_idx2 != len(xbin):
      weights = tuning_curve_CoM_coordinates[bin_idx1:bin_idx2+1] - CoM
      positions[i] = (weights * xbin[bin_idx1:bin_idx2+1]).sum() / weights.sum()

  return positions

def map_CoM_to_position_smooth_interp(xbin, tuning_curve_CoM_coordinates, xbin_edge_labels, sigma=1):
  """
  Maps center-of-mass coordinates back to actual positions using Gaussian smoothing and interpolation.

  Args:
      xbin: np.array, the bin centers (not edges).
      tuning_curve_CoM_coordinates: np.array, the center-of-mass coordinates for each tuning curve.
      xbin_edge_labels: np.array, the bin edge labels (not center).
      sigma: float, the sigma parameter for Gaussian smoothing.

  Returns:
      positions: np.array, the estimated positions corresponding to the input center-of-mass values.
  """
  smoothed_CoM = gaussian_filter1d(tuning_curve_CoM_coordinates, sigma)
  positions = np.full_like(tuning_curve_CoM_coordinates, np.nan)
  for i, CoM in enumerate(smoothed_CoM):
    # Find interpolation bin and check validity
    bin_idx = (xbin_edge_labels <= CoM).sum() - 1
    if bin_idx < len(xbin) - 1:
      # Perform linear interpolation between neighbors
      interpolator = interp1d(xbin_edge_labels[bin_idx:bin_idx+2], xbin[bin_idx:bin_idx+2], kind="linear")
      positions[i] = interpolator(CoM)

  return positions



In [ ]:

xbin_edge_labels = np.arange(len(ratemap.xbin)) # the index range spanning all x-bins
xbin = deepcopy(ratemap.xbin)

tuning_curve_CoM_coordinates = deepcopy(ratemap.peak_tuning_curve_center_of_masses)

assert np.all(np.diff(xbin) > 0)


# map_CoM_to_position_interp_array(xbin, tuning_curve_CoM_coordinates, xbin_edge_labels)

test_CoM_df = pd.DataFrame({'simple_interp': np.interp(tuning_curve_CoM_coordinates, xp=xbin_edge_labels, fp=xbin),
							'interp': map_CoM_to_position_interp_array(xbin, tuning_curve_CoM_coordinates, xbin_edge_labels),
                        #    'inv_weighted_avg': map_CoM_to_position_weighted_average(xbin, tuning_curve_CoM_coordinates, xbin_edge_labels),
                           'smooth_interp': map_CoM_to_position_smooth_interp(xbin, tuning_curve_CoM_coordinates, xbin_edge_labels, sigma=0.1)})
test_CoM_df
# xbin: np.array([36.5862, 40.3916, 44.197, 48.0025, 51.8079, 55.6133, 59.4187, 63.2241, 67.0295, 70.835, 74.6404, 78.4458, 82.2512, 86.0566, 89.862, 93.6675, 97.4729, 101.278, 105.084, 108.889, 112.695, 116.5, 120.305, 124.111, 127.916, 131.722, 135.527, 139.332, 143.138, 146.943, 150.749, 154.554, 158.36, 162.165, 165.97, 169.776, 173.581, 177.387, 181.192, 184.997, 188.803, 192.608, 196.414, 200.219, 204.025, 207.83, 211.635, 215.441, 219.246, 223.052, 226.857, 230.662, 234.468, 238.273, 242.079, 245.884, 249.69])
# tuning_curve_CoM_coordinates: np.array([22.3504, 33.5163, 36.219, 32.3855, 23.8594, 24.6407, 10.0726, 37.7695, 5.75679, 32.0758, 9.31427, 51.8735, 34.7201, 18.7215, 24.9917, 4.85872, 5.76916, 21.6332, 39.7348, 39.1924, 16.6196, 12.1798, 21.817, 15.8822, 7.94376, 46.0885, 24.9744, 18.708, 32.783, 46.3567, 40.9617, 30.5369, 47.7288, 19.5929, 37.7068, 50.7682, 18.0797, 30.9646, 18.4351, 26.3567, 46.3524, 9.01655, 42.9213, 33.1472, 7.45477, 11.561, 14.1151, 26.0069, 20.4058, 47.9163, 28.6427])
# xbin_edge_labels: np.array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56])